In [27]:
%load_ext autoreload
%autoreload 2
from balatro import Card, Blind_Manager, Deck_Manager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [317]:
deck_m = Deck_Manager()
blind_m = Blind_Manager()

In [ ]:
blind_m.blind_tracker(deck_m)

In [417]:
deck_m.draw_cards_and_get_best_hand()

Pair: ['2h', '2s']


{'score': 28, 'hand_type': 'Pair', 'cards_played': 2, 'discards_used': 0}

In [411]:
deck_m.shuffle_deck()